In [18]:
import datetime
import pandas as pd
from pandas.tseries.offsets import BDay as workday
from selenium.webdriver.support.ui import Select


In [19]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.expected_conditions import presence_of_element_located

In [20]:
login = dict()

In [21]:
with open('foxfundingemail.txt') as file:
    email = file.read()
    login['email']=str(email)
    
with open('foxfundingpw.txt') as pw_file:
    password = pw_file.read()
    login['password']=str(password)

In [22]:
beginning_url = "https://foxfundingportal.com/users/sign_in"

In [23]:
path = r"C:\Users\User\Dropbox (Fox)\Hardly Used\Coding\Python\ChromeDriver\chromedriver.exe"

In [24]:
driver = webdriver.Chrome(executable_path=path)

driver.get(beginning_url)   

driver.find_element_by_id("user_email").send_keys(login['email'])
driver.find_element_by_id("user_password").send_keys(login['password'])
driver.find_element_by_css_selector("input[type=submit]").click()


driver.get('https://foxfundingportal.com/deals')    

In [25]:
if driver:
    print('Success!')
else:
    print('An error has occurred.')

Success!


In [26]:
today = pd.datetime.today()
yesterday = (today-workday(1)).strftime('%m/%d/%Y')
from_date = "07/01/2019"
to_date = today.strftime('%m/%d/%Y')

def filter_set_up(driver):
    # Javascript remove disabled part of Underwriter field
    driver.execute_script("var selection = document.getElementById('underwriter');if(selection!==null){selection.removeAttribute('disabled');}")
    Select(driver.find_element_by_id("underwriter")).select_by_value('8')
    driver.find_element_by_id("from_created_date").send_keys(from_date)
    driver.find_element_by_id("to_created_date").send_keys(to_date)
    driver.find_element_by_id("search_name").send_keys(" ")
    driver.find_element_by_css_selector("input[type=submit]").click()

filter_set_up(driver)
to_date

'10/17/2019'

In [27]:
select_table_headers = driver.find_elements_by_css_selector("body > main > div > div > section > div > div.deal_panel > form > div.table_scroll_wrapper > div > table > thead > tr > th")
table_headers = list()
for elem in range(len(select_table_headers)-1):
    table_headers.append(select_table_headers[elem].text)
table_headers.append('Iso RM')

table_headers[9] = 'Renew (Y/N)'

table_headers

['Deals',
 'Date Submitted',
 'Status',
 'Note',
 'Funded Amount',
 'Term',
 'Monthly Volume',
 'FICO',
 'Pos',
 'Renew (Y/N)',
 'Iso',
 'Sales Rep',
 'Underwriter',
 'Iso RM']

In [28]:
MAX_PAGE_NUMBER = int(driver.find_element_by_css_selector("div.table_scroll_wrapper > div > div > div > a:nth-last-child(2)").text)

print(MAX_PAGE_NUMBER)

123


In [29]:
# underwriters_select = Select(driver.find_element_by_id('underwriter'))
# underwriters = list()
# for x in underwriters_select.options:
#     underwriters.append(x.text)
# underwriters

In [30]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def page_cycle(driver):
    WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located)
    driver.find_element_by_css_selector("div.table_scroll_wrapper > div > div > div > a:nth-last-child(1)").click()


In [31]:
filter_set_up(driver)

In [32]:
df = pd.read_html(driver.page_source)[0]

In [33]:
df.columns = table_headers

In [39]:
page_cycle(driver)

In [17]:
df_rest = pd.read_html(driver.page_source)[0]
df_rest.columns = table_headers
df = pd.concat([df,df_rest], join_axes=[df.columns], ignore_index=True)

AttributeError: 'NoneType' object has no attribute 'click'

In [ ]:
df

In [ ]:
df.info()

In [ ]:
print(df.Status.unique())
approved_statuses = ['Approved', 'Contracts Ba...', 'Contract Sent', 'Soft Approval', 'Ready to Fund']
declined_statuses = ['Negative Bal...', 'Declined', 'Bad Credit', 'Too Small', 'No Room', 'Previous Def...','Lost Deal', 'Merchant Dec...', 'Declined Pre...','Too Few Depo...', 'SIC Code']
funded_statuses = ['Funded']
submission_statuses = ['New', 'Need Info / ...']

In [ ]:
def status_groups(g):
    if g in approved_statuses:
        return "approved"
    elif g in declined_statuses:
        return "declined"
    elif g in funded_statuses:
        return "funded"
    else:
        return "submission"
    
df['group_status'] = df['Status'].apply(status_groups)
df.columns = df.columns.str.replace(' ','_')
print(df.columns)
df.Funded_Amount = df.Funded_Amount.replace('[\$,]', '', regex=True).astype(float)

In [ ]:
total_in_submissions = len(df.loc[df.group_status=='submission'])
total_declined = len(df.loc[df.group_status=='declined'])
total_approved = len(df.loc[df.group_status=='approved'])
total_funded_today = len(df.loc[df.group_status=='funded'])

submission_metric = total_in_submissions / len(df)
declined_metric = total_declined / len(df)
approved_metric = total_approved / len(df)
funded_metric = total_funded_today / len(df)

print(total_in_submissions, '\n',total_approved,'\n',total_declined,'\n',total_funded_today,'\n')
print(submission_metric, '\n',approved_metric,'\n',declined_metric,'\n',funded_metric,'\n')

# df.loc[(df.Underwriter=='Dave sneiderman') & (df.group_status=='approved'), 'Iso'].value_counts()
# df.loc[(df.Underwriter=='Dave sneiderman') & (df.group_status=='declined'), 'Iso'].value_counts()
# df.loc[(df.Underwriter=='Dave sneiderman') & (df.group_status=='funded')]
# df.loc[(df.Underwriter=='Dave sneiderman'), 'Iso'].value_counts()
# iso_counts = df.groupby(['Underwriter', 'Iso', 'group_status'])['Underwriter', 'Iso', 'group_status','Funded_Amount']
# iso_counts['total'] = iso_counts.approved + iso_counts.declined + iso_counts.funded + iso_counts.submission
# iso_counts['%approved'] = iso_counts.approved / iso_counts.total
# iso_counts['%declined'] = iso_counts.declined / iso_counts.total
# iso_counts['%funded'] = iso_counts.funded / iso_counts.total
# iso_counts['%submission'] = iso_counts.submission / iso_counts.total

In [ ]:
# Sum of all monies ($USD) approved yesterday
print(sum(df.loc[df['group_status']=='approved']['Funded_Amount']))
    
print('-----------------------------------------------')    
# Sum of all monies ($USD) same-day funded yesterday    
print(sum(df.loc[df['group_status']=='funded']['Funded_Amount']))
      

In [ ]:
df.Iso.unique()

In [ ]:
df.groupby(['Underwriter','Iso'])['Funded_Amount']

In [ ]:
for iso in df.Iso.unique():
    print(iso,
          sum(df.loc[(df['Iso']==iso) & (df['group_status']=='submission')].count()),
          sum(df.loc[(df['Iso']==iso) & (df['group_status']=='declined')].count()),
          sum(df.loc[(df['Iso']==iso) & (df['group_status']=='approved')].count()),
          sum(df.loc[(df['Iso']==iso) & (df['group_status']=='funded')].count()),
          sum(df.loc[(df['Iso']==iso) & df['group_status'].isin(['approved','funded'])]['Funded_Amount']))

In [ ]:
# # Database stuff
# import sqlalchemy as db
# mysql_password = ''
# with open('mysqlpw.txt') as file:
#     mysql_password = file.read()
# engine = db.create_engine('mysql://ryan:'+ mysql_password +'@localhost:3306/daily_reports')


In [ ]:
# # Excel stuff
# excel_table_names = dict()
# for underwriter in df['Underwriter'].unique():
#     excel_table_names[str(underwriter)] = str(underwriter) + ' Report '+ today.strftime('%m.%d.%Y') + '.xlsx'
# print(excel_table_names)




In [ ]:
# df.to_sql(
#     name='funding_solutions',
#     con=engine,
#     index=False,
#     if_exists='append'
# )

In [ ]:
with open('rows.csv','w') as file:
    file.write(str(df))

In [ ]:
# query = r"C:\Users\User\Downloads\underwriting-20190703T205345Z-001\underwriting\NEW_DEALS_YESTERDAY.sql"
# with open(query) as file:
#     pd.read_sql_query(file.read(), engine)
